In [8]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from itertools import islice

nltk.download('stopwords')

df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

[nltk_data] Downloading package stopwords to /home/camix/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
df_train['titulo'] = df_train['titulo'].fillna('vacio')
df_train['descripcion'] = df_train['descripcion'].fillna('vacio')

In [52]:
def generar_palabras_no_queridas(arr=None):
    '''Retorna un set con las palabras a excluir, arr, es un campo opcional para excluir tambien 
    las palabras en ese arreglo'''
    stop_words = set(stopwords.words('spanish'))
    puntuacion = [',', '.', '!', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-', '_', '\'', '\"', 
                  '(', ')', '[', ']', '{', '}', 'i', 'p', 'br', '<', '>', 'col.', '\\', '/', '$', 'm2', 
                  '..', '...', 'ii', '#', ';', ':', 'col']
    
    for x in puntuacion:
        stop_words.add(x)
    if arr:
        for x in arr:
            stop_words.add(x)
    return stop_words

def generar_token(textos):
    reviews = textos.str.cat(sep=' ')
    tokens = word_tokenize(reviews)
    stop_words = generar_palabras_no_queridas()
    tokens = [w for w in tokens if not w in stop_words]
    return tokens

In [54]:
def tf_idf (df,cols):
    cvec = CountVectorizer()
    tokens = generar_token(df[cols])
    cvec.fit(tokens)
    cvec_counts = cvec.fit_transform(tokens)
    transformer = TfidfVectorizer()
    transformed_weights = transformer.fit_transform(tokens)
    weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
    weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': weights}).set_index('term')
    return weights_df

In [59]:
def contador_mas_frecuentes(df, df_idf, col):
    contadores = []
    for texto in df[col].values:
        palabras_del_df = word_tokenize(texto)
        contador = 0
        for palabra in palabras_del_df:
            if palabra in df_idf.index:
                contador = contador + df_idf.loc[palabra].weight
        contadores.append(contador)
    return contadores

In [56]:
df_titulo = tf_idf(df_train, 'titulo')
df_descripcion = tf_idf(df_train, 'descripcion')

In [64]:
df_train['idf_titulo'] = contador_mas_frecuentes(df_train, df_titulo, 'titulo')

In [66]:
df_train['idf_descripcion'] = contador_mas_frecuentes(df_train, df_descripcion, 'descripcion')

In [84]:
palabras_importantes = ['balcon', 'estacionamiento','niveles','pent', 'vista', 'mar', 'lavanderia', 'patio', 'bien', 'hermosa', 'hermoso', 'jardin', 'vigilancia', 'preciosa', 'excelente', 'equipada', 'lujo', 'garage', 'piscina', 'pileta', 'privada', 'exclusiva', 'penthouse', 'facil']

def contador_mas_importantes(df, col):
    contadores = []
    for texto in df[col].values:
        palabras_del_df = word_tokenize(texto)
        contador = 0
        for palabra in palabras_del_df:
            if palabra in palabras_importantes:
                contador = contador + 1
        contadores.append(contador)
    return contadores

In [85]:
df_train['peso_titulo'] = contador_mas_importantes(df_train, 'titulo')

In [86]:
df_train['peso_descripcion'] = contador_mas_importantes(df_train, 'descripcion')

In [87]:
df_train[['descripcion', 'peso_descripcion', 'precio']].sort_values('precio', ascending = False)

,descripcion,peso_descripcion,precio
174276,excelente oficina en venta en torre world trad...,3,12525000.0
209858,condominio tipo mexicano moderno de 3 casas ub...,0,12521000.0
81715,proyecto de imanol legorreta y carlos duclaud ...,3,12520000.0
192956,"edificio de 5 niveles con 9 departamentos, ofr...",3,12516000.0
232457,¡excelente oportunidad de inversión! nosotros ...,2,12515000.0
75131,<p>edificio en donde la prioridad es aprovecha...,0,12500000.0
24470,residencia en venta en la zona más exclusiva d...,4,12500000.0
173245,"se vende casa en copilco universidad, coyoacán...",1,12500000.0
139897,"se vende hermosa casa sola en calle privada, c...",5,12500000.0
131181,terreno 1000 mts2 \nconstrucción 250 mts + rem...,3,12500000.0
